# Imports

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import pytz
from datetime import datetime as dt
from hbt_tools import utils_hbt as uh
from hbt_tools import plots as hp
from pathlib import Path
from plotly.subplots import make_subplots

# Functions

In [ ]:
def poleni(h, B, mu):
    return 2/3 * mu * (2 * 9.81)**(1/2) * B * (h)**(3/2)
#-------------------------------------------------------------------------------

def plot_pw(ts_pw, ts_rain, names=['PW', 'rain']):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=ts_pw.index,
        y=ts_pw.iloc[:,0],
        name=names[0],
        line=dict(color='red', width=1),
        yaxis="y1"
        ))
    
    fig.add_trace(go.Bar(
        x=ts_rain.index,
        y=ts_rain.iloc[:,0],
        name=names[1],
        marker_color='blue', 
        yaxis="y2"
        ))

    fig.update_layout(
        yaxis=dict(
            title="Niveau",
            titlefont=dict(color="red"),
            tickfont=dict(color="red"),
            anchor="free",
            side="left",
        ),
        yaxis2=dict(
            title="Regenmenge [mm/10 min]",
            titlefont=dict(color="blue"),
            tickfont=dict(color="blue"),
            anchor="free",
            overlaying="y",
            side="right",
        )
    )

    return fig
#-------------------------------------------------------------------------------

def filter_ts(ts, date_from, date_to):
    ts = ts[(ts.index>=date_from) & (ts.index<date_to)]

    return ts
#-------------------------------------------------------------------------------

def del_values(ts, val):
    mask = ts.iloc[:,0]==val
    ts_out = ts.drop(ts[mask].index)
    return ts_out
#-------------------------------------------------------------------------------

# Parameters

## General

In [ ]:
dt_format = '%d.%m.%Y %H:%M'
save_dir = Path(r"Q:\Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Plots")
date_from = dt(2021, 9, 29, tzinfo=pytz.timezone('Europe/Zurich'))
date_to = dt(2022, 10, 1, tzinfo=pytz.timezone('Europe/Zurich'))

## Messeinheiten

In [ ]:
measurements_units = {
    '432': { # data until 07.06!
        'name': 'Niveau Pumpensumpf PW Hirsen',
        'units': 'cm',
        'color': '',
        'y_axis': 'y1',
    },

    '457': { # data until 06.04!
        'name': 'Niveau Pumpensumpf PW Sonnenbergstrasse',
        'units': 'm',
        'color': '',
        'y_axis': 'y1',
    },

    '458': { # OK
        'name': 'Niveau Einlaufkammer Absetzbecken',
        'units': 'm',
        'color': '',
        'y_axis': 'y1',
    },

    '459': { # OK
        'name': 'RFB 1 - Niveau 1',
        'units': 'm',
        'color': '',
        'y_axis': 'y1',
    },

    '460': { # OK
        'name': 'RFB 1 - Niveau 2',
        'units': 'bar',
        'color': '',
        'y_axis': 'y1',
    },

    '461': { # OK
        'name': 'RFB 2 - Niveau 1',
        'units': 'm',
        'color': '',
        'y_axis': 'y1',
    },

    '462': { # no data!
        'name': 'RFB 2 - Niveau 2',
        'units': 'm',
        'color': '',
        'y_axis': 'y1',
    },

    '463': { # no data!
        'name': 'Trübung Auslaufbauwerk RB Mühlestrasse',
        'units': 'FNU',
        'color': '',
        'y_axis': 'y3',
    },

    '464': { # OK
        'name': 'Trübung Ablauf Absetzbecken',
        'units': 'FNU',
        'color': '',
        'y_axis': 'y3',
    },

    '465': { # OK
        'name': 'Niveau Auslaufbauwerk RB Mühlestrasse',
        'units': 'cm',
        'color': '',
        'y_axis': 'y1',
    },

    '466': { # OK
        'name': 'Durchflussmessung Auslauf SABA',
        'units': 'L/s',
        'color': 'blue',
        'y_axis': 'y2',
    }
}

# Rain Data

In [ ]:
path_rain = Path(r"Q:\Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Regendaten\LUZ_2020 bis 2023_data.csv")

rain = uh.read_raindata_meteoschweiz(path_rain, abbr=True)
rain = rain[(rain.index>=date_from) & (rain.index<date_to)]

# Pumpwerke

## PW Sonnenbergstrasse

In [ ]:
# Read data
id = 457
lvl_sonnenberg = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)
lvl_sonnenberg = filter_ts(lvl_sonnenberg, date_from, date_to)

In [ ]:
fig = hp.ply_2y(lvl_sonnenberg, rain, names=['Niveau PW', 'Regen LUZ'],
    xlabel='Zeit', ylabels=['Niveau [m]', 'Regen [mm/10min]'],
    colors=['red', 'blue'])

In [ ]:
fig.update_layout(
    title='PW Sonnenbergstrasse', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)))

h_ül = 4.5 # m, im Pumpensumpf
fig.add_hline(y=h_ül, line_width=1, line_color='black')
fig.add_annotation(x=lvl_sonnenberg.index[0], y=h_ül,text=f'{h_ül} m',
    font=dict(size=20, color='black'))
fig

In [ ]:
save_path = save_dir / 'Niveau_PW-Sonnenbergstrasse.html'
fig.write_html(save_path)

## PW Hirsen

In [ ]:
# Read data
id = 432
lvl_hirsen = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)
lvl_hirsen = filter_ts(lvl_hirsen, date_from, date_to)
lvl_hirsen = lvl_hirsen / 100 # cm -> m

In [ ]:
fig = hp.ply_2y(lvl_hirsen, rain, names=['Niveau PW', 'Regen LUZ'],
    xlabel='Zeit', ylabels=['Niveau [m]', 'Regen [mm/10min]'],
    colors=['red', 'blue'])

In [ ]:
fig.update_layout(
    title='PW Hirsen', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)))

h_ül = 3.95 # m, im Pumpensumpf
fig.add_hline(y=h_ül, line_width=1, line_color='black')
fig.add_annotation(x=lvl_hirsen.index[0], y=h_ül,text=f'{h_ül} m',
    font=dict(size=20, color='black'))
fig

In [ ]:
save_path = save_dir / 'Niveau_PW-Hirsen.html'
fig.write_html(save_path)

# SABA

## ASB

### Niveau Einlaufkammer

In [ ]:
id = 458
asb1 = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)

In [ ]:
fig = hp.ply_2y(asb1, rain, names=['Niveau Einlauf ASB', 'Regen LUZ'],
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Regen [mm/10min]'],
    colors=['red', 'blue'])

In [ ]:
fig

In [ ]:
save_path = save_dir / 'Niveau_Einlauf-ASB.html'
fig.write_html(save_path)

### Niveau Becken

In [ ]:
id = 465

# Niveau
asb2 = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)
asb2 = asb2 - 105 # cm, 1.05 Messwert bei ansprigngen des Wehres/Entlastung (Christian)

# Durchfluss
B = 2.75 # der Wert ist aus dem Schalungsplan
mu = 0.6
q_asb = poleni(B=B, mu=mu, h=asb2/100) * 1000 # l/s
q_asb.fillna(0, inplace=True)

In [ ]:
fig = hp.ply_2y(asb2, rain, names=['Niveau ASB', 'Regen LUZ'], 
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Regen [mm/10min]'],
    colors=['red', 'blue'])

In [ ]:
fig.update_layout(
    title='Absetzbecken', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)))

fig

In [ ]:
save_path = save_dir / 'Niveau_ASB.html'
fig.write_html(save_path)

### Zufluss Becken

In [ ]:
fig = hp.ply_2y(asb2, q_asb, names=['Niveau ASB', 'Zufluss ASB'], 
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Zufluss [l/s]'],
    colors=['red', 'cyan'])

In [ ]:
fig.update_layout(
    title='Zufluss Absetzbecken', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)))

fig

In [ ]:
save_path = save_dir / 'Zufluss_ASB.html'
fig.write_html(save_path)

## RFB

### RFB 1

In [ ]:
id = 459
rfb1 = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)

In [ ]:
# Clean data
rfb1 = (rfb1 - 1.73) * -1 * 100
rfb1 = del_values(rfb1, 73)

In [ ]:
fig = hp.ply_2y(rfb1, rain, names=['Niveau RFB 1', 'Regen LUZ'],
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Regen [mm/10min]'],
    colors=['red', 'blue'])

In [ ]:
fig.update_layout(
    title='Niveau RFB 1', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)))

h_ül = 67 # cm - gemäss Fotos Begehung
fig.add_hline(y=h_ül, line_width=1, line_color='black')
fig.add_annotation(x=rfb1.index[0], y=h_ül,text=f'{h_ül} cm',
    font=dict(size=20, color='black'))
fig

In [ ]:
save_path = save_dir / 'Niveau_RFB1.html'
fig.write_html(save_path)

### RFB 2

In [ ]:
id = 461
rfb2 = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)

In [ ]:
# Reframe data points
rfb2_ = (rfb2 - 0.92) * -1 * 100
rfb2_ = del_values(rfb2_, 92)

In [ ]:
fig = hp.ply_2y(rfb2_, rain, names=['Niveau RFB 2', 'Regen LUZ'],
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Regen [mm/10min]'],
    colors=['red', 'blue'])

In [ ]:
fig.update_layout(
    title='Niveau RFB 2', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)))

h_ül = 59 # cm - gemäss Fotos Begehung
fig.add_hline(y=h_ül, line_width=1, line_color='black')
fig.add_annotation(x=rfb1.index[0], y=h_ül,text=f'{h_ül} cm',
    font=dict(size=20, color='black'))
fig

In [ ]:
save_path = save_dir / 'Niveau_RFB2.html'
fig.write_html(save_path)

In [ ]:
# Read data
lvl1 = 459
lvl2 = 460
lvl_asb = 458


dt_start = '12.10.2022 10:00:00'
dt_end = '01.11.2022 00:00:00'

ts_lvl1 = uh.hbtdb_get_data(lvl1, dt_start, dt_end)
ts_lvl2 = uh.hbtdb_get_data(lvl2, dt_start, dt_end)
ts_asb = uh.hbtdb_get_data(lvl_asb, dt_start, dt_end)

# Shift and invert data
ts_lvl1 = (ts_lvl1 - 1.73) * -1 * 100
ts_asb = (ts_asb - 4.43) * -1

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts_lvl1.index, y=ts_lvl1['wert'],
    name='Niveau RFB1'))
fig.add_trace(go.Scatter(x=ts_asb.index, y=ts_asb['wert'],
    name='Niveau ASB'))
fig.update_xaxes(title='Zeit')
fig.update_yaxes(title='Niveau [cm]')
fig.update_layout(title='Niveau RFB1')

save_path = save_dir / 'rfb1.html'
fig.write_html(save_path)

In [ ]:
dates = ['2022-06-14', '2022-06-27', '2022-07-04']
colors = ['red', 'green']

fig = make_subplots(specs=[[{"secondary_y": False}]])
for i, id in enumerate(ids):
    fig.add_trace(go.Scatter(x=data[id].index, y=data[id]['wert'],
        name=measurements_units.loc[id]['name'], marker_color=colors[i]))

fig.add_vrect(x0=dates[0], x1=dates[1], row="all", col=1, 
    annotation_text="Beschickung RFB 1", annotation_position="top left",
    fillcolor=colors[0], opacity=0.2, line_width=1, line_color='black')

fig.add_vrect(x0=dates[1], x1=dates[2], row="all", col=1, 
    annotation_text="Beschickung RFB 2", annotation_position="top left",
    fillcolor=colors[1], opacity=0.2, line_width=1, line_color='black')

ax = -30
ay = -30
for i, rev in enumerate(rain_ev1_start):
    fig.add_annotation(x=rev, y=data['459'].loc[rev].values[0],
        text=f'Ereignis {i+1}', showarrow=True, ax=ax, ay=ay)

for i, rev in enumerate(rain_ev2_start):
    if i == len(rain_ev2_end) - 1:
        ax = 0
        ay = -60
    fig.add_annotation(x=rev, y=data['461'].loc[rev].values[0],
        text=f'Ereignis {n1+1+i}', showarrow=True, ax=ax, ay=ay)

fig.update_layout(
    title='Niveaus SABA',
    xaxis_title="Datum",
    yaxis_title="Niveau [m]",
)

file_path = save_dir / ('Niveaus-SABA.html')
fig.write_html(file_path)